# REST-ASMR: Statistical Analysis & Technical Validation

>  **PREREQUISITE:** This notebook provides the visual and statistical validation for the REST-ASMR dataset. Before running any cells below, ensure to execute the backend orchestrator by running `!python main.py`
> 
> The orchestrator processes the raw signals, extracts the deep learning features, and populates the `./data/features/` and `./data/` directories with the necessary artifacts that this notebook analyzes.

---

### Behavioral Inter-Subject Agreement (ISA)

**Methodology:**
To verify that the ASMR tingling sensations were stimulus-locked with a cross-participant consensus, a Leave-One-Out Inter-Subject Agreement (LOO-ISA) was calculated using Fisher's Z-transformed Pearson's $r$ and bootstrap resaampling to ascertain if the group agreement was significantly greater than zero.

This code outputs the 95% Confidence Intervals (CIs) and $p$-values reported in the manuscript.

In [ ]:
import numpy as np
import glob
import os
import scipy.stats
import warnings

warnings.filterwarnings("ignore")

LABEL_FOLDER = "./data/features"

def calculate_bootstrapped_isa():
    
    print(" BOOTSTRAPPED BEHAVIORAL INTER-SUBJECT AGREEMENT (ISA) ")
   
    
    label_files = glob.glob(os.path.join(LABEL_FOLDER, "y_*.npy"))
    if not label_files:
        print("No label files found.")
        return

    all_vids = ["vid1", "vid2", "vid3", "vid4", "vid5", "vid6", "vid7", "vid8"]
    video_labels = {vid: [] for vid in all_vids}
    
    for f in label_files:
        filename = os.path.basename(f)
        vid = filename.replace(".npy", "").split("_")[2]
        video_labels[vid].append(np.load(f))
        
    for condition, vids in [("ASMR (Tingle)", ["vid1", "vid2", "vid3", "vid4"]), 
                            ("Nature (Pleasantness)", ["vid5", "vid6", "vid7", "vid8"])]:
        print(f"\n  {condition}   ")
        
        for vid in vids:
            arrays = video_labels[vid] 
            if len(arrays) < 2:
                print("Wrong")
                continue
            
                
            
            min_len = min(len(arr) for arr in arrays)
            stacked_arrays = np.vstack([arr[:min_len] for arr in arrays])
            
            num_subjects = stacked_arrays.shape[0]
            subject_z_scores = []
            
           
            for i in range(num_subjects):
                target_sub = stacked_arrays[i]
                other_subs_idx = [j for j in range(num_subjects) if j != i]
                mean_others = np.mean(stacked_arrays[other_subs_idx], axis=0)
                
                if np.std(target_sub) > 0 and np.std(mean_others) > 0:
                    r, _ = scipy.stats.pearsonr(target_sub, mean_others)
                    if not np.isnan(r):
                        subject_z_scores.append(np.arctanh(r))
                        
            if not subject_z_scores:
                print(f"  > {vid.upper()}: Insufficient variance for ISA.")
                continue

            
            np.random.seed(42) 
            n_iterations = 10000
            z_array = np.array(subject_z_scores)
            
            
            bootstrap_means = [np.mean(np.random.choice(z_array, size=len(z_array), replace=True)) 
                               for _ in range(n_iterations)]
            
            
            mean_z = np.mean(bootstrap_means)
            mean_r = np.tanh(mean_z)
            
            
            ci_lower_z, ci_upper_z = np.percentile(bootstrap_means, [2.5, 97.5])
            ci_lower_r = np.tanh(ci_lower_z)
            ci_upper_r = np.tanh(ci_upper_z)
            
         
            p_val = np.sum(np.array(bootstrap_means) <= 0) / n_iterations
            
            
            sig_star = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
            
            print(f"  > {vid.upper()} Mean ISA (r) : {mean_r:.4f} {sig_star}")
            print(f"      95% CI       : [{ci_lower_r:.4f}, {ci_upper_r:.4f}]")
            print(f"      p-value      : {p_val:.4f} (Bootstrapped)")



if __name__ == "__main__":
    calculate_bootstrapped_isa()

### Physiological Signal Integrity & Cardiovascular Response


**Methodology:**
This section validates the biological efficacy of the stimuli by assessing the subject-wise parasympathetic cardiovascular deceleration associated with the ASMR state. It utilizes a 5-s sliding window approach with Spearman rank correlation and paired-samples t-test to stastically quantify the difference in ASMR vs. Nature induced relaxation.

This code outputs the statistical significance ($p$-value) and generates the manuscript **Figure**.

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import scipy.stats
import scipy.signal
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

RAW_PPG_FOLDER = "./data/ppg"
LABEL_FOLDER = "./data/features" 
LOG_FOLDER = "./data/log"
WINDOW_SEC = 5.0   
STEP_SEC = 1.0     

def load_raw_ppg(filepath):
   
    encodings = ['shift_jis', 'utf-8', 'cp1252']
    for enc in encodings:
        try:
            df = pd.read_csv(filepath, sep='\t', skiprows=8, encoding=enc, on_bad_lines='skip')
            if df.shape[1] >= 3:
                df = df.iloc[:, [0, 2]] 
                df.columns = ['Time_Min', 'Val']
                df = df[pd.to_numeric(df['Time_Min'], errors='coerce').notnull()]
                df['Time_Sec'] = df['Time_Min'].astype(float) * 60.0
                return df
            else:
                print("Wrong shape")
        except:
            print("Wrong!")
            continue
    return None

def get_window_features(chunk, fs=2000.0):
   
  
    sos = scipy.signal.butter(2, [0.5, 4.0], btype='bandpass', fs=fs, output='sos')
    filtered = scipy.signal.sosfiltfilt(sos, chunk)
    
   
    peaks, _ = scipy.signal.find_peaks(filtered, distance=int(fs*0.35), prominence=0.5)
  
    
    if len(peaks) < 4: return np.nan, np.nan 
    
    
    rr_intervals = np.diff(peaks) / fs * 1000.0   
    
    
    valid_rr = rr_intervals[(rr_intervals > 300) & (rr_intervals < 1300)]
    
    if len(valid_rr) < 3: return np.nan, np.nan
    
    hr = 60000.0 / np.mean(valid_rr)  
    rmssd = np.sqrt(np.mean(np.diff(valid_rr)**2))
    
    return hr, rmssd

def compare_asmr_nature_physiology():
    print(" STARTING SLIDING WINDOW COMPARISON (ASMR vs NATURE) ")
    
   
    timeline_map = {} 
    log_files = glob.glob(os.path.join(LOG_FOLDER, "*.log"))
    for log_f in log_files:
        try:
            sub_id = int(os.path.basename(log_f).split('-')[0])
            with open(log_f, 'r', encoding='utf-8', errors='ignore') as f: lines = f.readlines()
            header_idx = next(i for i, l in enumerate(lines) if l.startswith("Subject"))
            ldf = pd.read_csv(log_f, sep='\t', skiprows=header_idx)
            vids = ldf[ldf['Event Type'] == 'Video'].reset_index(drop=True)
            for i, row in vids.iterrows():
                dur = 55.7 if (i==0) else 60.0
                timeline_map[(sub_id, row['Code'])] = (row['Time']/10000.0, dur)
        except: 
            print("Wrong!!")
            continue

    results = []
    subjects = sorted(list(set([k[0] for k in timeline_map.keys()])))
    
    for sub in tqdm(subjects, desc="Analyzing Subjects"):
        raw_path = os.path.join(RAW_PPG_FOLDER, f"{str(sub).zfill(3)}_csv.txt")
        if not os.path.exists(raw_path): 
            print("WRONG")
            continue
        
        df_ppg = load_raw_ppg(raw_path)
        if df_ppg is None: 
            print("WRONG")
            continue
        
        
        data_buckets = {
            'ASMR': {'hr': [], 'hrv': [], 'rating': []},
            'Nature': {'hr': [], 'hrv': [], 'rating': []}
        }
        
        all_videos = ["vid1", "vid2", "vid3", "vid4", "vid5", "vid6", "vid7", "vid8"]
        
        for vid in all_videos:
            if (sub, vid) not in timeline_map: continue
            
           
            category = 'Nature' if vid in ["vid5", "vid6", "vid7", "vid8"] else 'ASMR'
            
            start_sec, dur_sec = timeline_map[(sub, vid)]
            
           
            y_path = os.path.join(LABEL_FOLDER, f"y_{sub}_{vid}.npy")
            if not os.path.exists(y_path): continue
            y_data = np.load(y_path)
            
            
            curr_time = start_sec + 5.0
            end_time = start_sec + dur_sec
            
            while curr_time + WINDOW_SEC < end_time:
                
                mask = (df_ppg['Time_Sec'] >= curr_time) & (df_ppg['Time_Sec'] < curr_time + WINDOW_SEC)
                chunk = df_ppg.loc[mask, 'Val'].values
                
                
              
                rel_start = curr_time - (start_sec + 5.0)
                idx_start = int(rel_start * 10)    #as labels are in 10 hz and rel_start in s
                idx_end = int((rel_start + WINDOW_SEC) * 10)
                
                if idx_end <= len(y_data) and len(chunk) > 1000:
                    hr, hrv = get_window_features(chunk)
                    avg_rating = np.mean(y_data[idx_start:idx_end])
                    
                    if not np.isnan(hr) and not np.isnan(hrv):
                        data_buckets[category]['hr'].append(hr)
                        data_buckets[category]['hrv'].append(hrv)
                        data_buckets[category]['rating'].append(avg_rating)
                
                curr_time += STEP_SEC

       
        for cat in ['ASMR', 'Nature']:
            hrs = data_buckets[cat]['hr']
            hrvs = data_buckets[cat]['hrv']
            rats = data_buckets[cat]['rating']
            
            if len(hrs) > 10 and np.std(rats) > 0:
                r_hr, p_hr = scipy.stats.spearmanr(hrs, rats)
                r_hrv, p_hrv = scipy.stats.spearmanr(hrvs, rats)
                
                results.append({
                    'Subject': sub,
                    'Condition': cat,
                    'r_HR': r_hr,
                    'r_HRV': r_hrv,
                    'p_HR': p_hr
                })

    
    df_res = pd.DataFrame(results)
    
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
   
    sns.boxplot(x='Condition', y='r_HR', data=df_res, ax=ax, palette="Set2")
    sns.stripplot(x='Condition', y='r_HR', data=df_res, color='black', alpha=0.5, ax=ax)
    
    
    ax.set_title("Correlation: HR vs. Rating (ASMR vs Nature)", fontweight='bold')
    ax.set_ylabel("Spearman Correlation (r)")
    ax.axhline(0, color='red', linestyle='--')
    
   
    plt.tight_layout()
    #plt.savefig("asmr_hr_stat_test.svg")
    plt.show()
    
    asmr_hr = df_res[df_res['Condition']=='ASMR'].set_index('Subject')['r_HR']
    nat_hr = df_res[df_res['Condition']=='Nature'].set_index('Subject')['r_HR']
    
   
    common = pd.concat([asmr_hr, nat_hr], axis=1, join='inner')
    common.columns = ['ASMR', 'Nature']
    
    t_stat, p_val = scipy.stats.ttest_rel(common['ASMR'], common['Nature'])
    
    print("\n STATISTICAL COMPARISON (Paired T-Test) ")
    print(f"Mean HR Correlation (ASMR):   {common['ASMR'].mean():.4f}")
    print(f"Mean HR Correlation (Nature): {common['Nature'].mean():.4f}")
    print(f"Difference P-Value: {p_val:.4f}, t-stat: {t_stat} ")
    asmr_z = np.arctanh(common['ASMR'])
    nat_z = np.arctanh(common['Nature'])
    
   
    t_stat2, p_val2 = scipy.stats.ttest_rel(asmr_z, nat_z)
    print(f"Corrected Difference P-Value: {p_val2:.4f}, t-stat: {t_stat2} ")
    if p_val < 0.05:
        print("SIGNIFICANT DIFFERENCE found between physiological response to ASMR vs Nature.")
    else:
        print("No significant global difference in correlation magnitude.")

if __name__ == "__main__":
    compare_asmr_nature_physiology()

### Group-Level Temporal Agreement

**Visualization Methodology:**
This visualizes the temporal consensus of the ASMR tingling sensation across all active participants for the two highest-performing stimuli (Video 2 and Video 3). The figure displays **Mean Signal** with a solid line and the **Standard Error of Mean** using the shaded region.


This code outputs the manuscript Figure.

In [ ]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

LABEL_FOLDER = "./data/features"

def get_mean_timeseries(vid_target):
    label_files = glob.glob(os.path.join(LABEL_FOLDER, f"y_*_{vid_target}.npy"))
    if not label_files:
        return None, None, None
        
    subject_arrays = []
    for f in label_files:
        y_data = np.load(f)
        if np.max(y_data) > 0: 
            subject_arrays.append(y_data)

    if not subject_arrays: return None, None, None

    min_len = min(len(arr) for arr in subject_arrays)
    stacked_data = np.vstack([arr[:min_len] for arr in subject_arrays])
    
    mean_signal = np.mean(stacked_data, axis=0)
    sem_signal = np.std(stacked_data, axis=0) / np.sqrt(stacked_data.shape[0])
    time_axis = np.arange(min_len) / 10.0 
    
    return time_axis, mean_signal, sem_signal

def plot_dual_asmr_timeseries():
    
    
    t_vid2, mean_vid2, sem_vid2 = get_mean_timeseries("vid2")
    t_vid3, mean_vid3, sem_vid3 = get_mean_timeseries("vid3")
    
    if t_vid2 is None or t_vid3 is None:
        print("Data missing")
        return

    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
    sns.set_theme(style="whitegrid")
    
    
    axes[0].plot(t_vid2, mean_vid2, color='firebrick', linewidth=2.5, label='Mean Intensity')
    axes[0].fill_between(t_vid2, np.maximum(0, mean_vid2 - sem_vid2), mean_vid2 + sem_vid2, color='lightcoral', alpha=0.4, label='Standard Error')
    axes[0].set_title("A. Group-Level Temporal Agreement: Video 2", fontweight='bold')
    axes[0].set_ylabel("Mean Subjective Rating (0-3)", fontweight='bold')
    axes[0].set_xlabel("Trial Time (Seconds)", fontweight='bold')
    axes[0].set_ylim(0, 3.0)
    axes[0].legend(loc="upper left")

    
    axes[1].plot(t_vid3, mean_vid3, color='darkred', linewidth=2.5, label='Mean Intensity')
    axes[1].fill_between(t_vid3, np.maximum(0, mean_vid3 - sem_vid3), mean_vid3 + sem_vid3, color='indianred', alpha=0.4, label='Standard Error')
    axes[1].set_title("B. Group-Level Temporal Agreement: Video 3", fontweight='bold')
    axes[1].set_xlabel("Trial Time (Seconds)", fontweight='bold')
    axes[1].legend(loc="upper left")

    plt.tight_layout()
    #plt.savefig("Figure1_DualASMR_TimeSeries.svg", dpi=300)
    plt.show()

if __name__ == "__main__":
    plot_dual_asmr_timeseries()

### 5. Multimodal Predictive Modeling: Reproducibility Setup

**Methodology & Hardware Variance:**
Deep learning models trained on GPU hardware utilizing CuDNN backends exhibit  non-determinism due to parallelized atomic operations during backpropagation. To account for this fact and enable strict reproducibility of the manuscript's findings, this evaluation pipeline relies on serialized inference artifacts (`.pkl` files).

**Execution Modes:**
This data-loading block controls the artifact pipeline for the remainder of the notebook:
* `USE_PAPER_ARTIFACTS = True`: Loads the exact file (`paper_fusion_results_4_fold.pkl`) generated by the authors and utilized in the manuscript to reproduce all tables and $p$-values.
* `USE_PAPER_ARTIFACTS = False`: Loads dynamic artifacts (`fusion_results.pkl`) generated by a fresh run of the orchestrator to evaluate newly trained network weights.

In [ ]:
import pickle
import os


USE_PAPER_ARTIFACTS = True

if USE_PAPER_ARTIFACTS:
    file_path = './data/paper_fusion_results_4_fold.pkl'  #saved by the authors
    print("MODE: Reproducing Manuscript Results")
else:
    file_path = './data/fusion_results.pkl'            # generated now by a fresh run of the main orchestrator
    print("MODE: Evaluating Newly Trained Model")

try:
    with open(file_path, 'rb') as f:
        results = pickle.load(f)
    print(f"Successfully loaded {len(results)} trial sequences from '{file_path}'")
    
except FileNotFoundError:
    print(f"Error: '{file_path}' not found.")
    

### 5. Manuscript Performance Reconstruction 

**Methodology:**
 
To replicate the manuscript results, ensure that the above cell with "USE_PAPER_ARTIFACTS = True" was run.
This cell allows strict replication and accounts for the fact that Deep learning models trained on GPU hardware utilizing CuDNN backends exhibit inherent non-determinism due to parallelized atomic operations during backpropagation.

This section reconstructs the exact fold-by-fold performance metrics and the global aggregated summary (Means $\pm$ Standard Deviations) from the saved model inference artifacts reported. These metrics form the multimodal model results reported in the manuscript.

In [ ]:
def reconstruct_full_training_summary(test_results):
    print("  RECONSTRUCTING 4-FOLD METRICS FROM RESULTS  ")
    
   
    fold_vid_map = {
        0: ["vid1", "vid5"],
        1: ["vid2", "vid6"],
        2: ["vid3", "vid7"],
        3: ["vid4", "vid8"]
    }
    
    metrics_history = {
        'accuracy': [],
        'macro_f1': [],
        'class1_f1': [],
        'class0_f1': [],
        'nature_spec': [],
        'precision': [],
        'recall': []
    }
    
    for fold in range(4):
       
        fold_vids = fold_vid_map[fold]
        fold_trials = [tr for tr in test_results if tr['vid_name'] in fold_vids]
        
        if not fold_trials:
            print(f"Fold {fold+1} - No data found in results.")
            continue
            
        
        final_truths = np.concatenate([tr['truths'] for tr in fold_trials])
        final_preds = np.concatenate([tr['preds'] for tr in fold_trials])
     
        nature_trials = [tr for tr in fold_trials if tr['is_nature'] == 1]
        if nature_trials:
            nature_truths = np.concatenate([tr['truths'] for tr in nature_trials])
            nature_preds = np.concatenate([tr['preds'] for tr in nature_trials])
            fold_nat_spec = accuracy_score(nature_truths, nature_preds)
        else:
            fold_nat_spec = 1.0
            
        
        report_dict = classification_report(final_truths, final_preds, output_dict=True, zero_division=0)
        
        print(f"  > Fold {fold+1} Macro F1: {report_dict['macro avg']['f1-score']:.4f}")
        
       
        metrics_history['accuracy'].append(report_dict['accuracy'])
        metrics_history['macro_f1'].append(report_dict['macro avg']['f1-score'])
        metrics_history['class1_f1'].append(report_dict.get('1', {}).get('f1-score', 0))
        metrics_history['class0_f1'].append(report_dict.get('0', {}).get('f1-score', 0))
        metrics_history['precision'].append(report_dict['macro avg']['precision'])
        metrics_history['recall'].append(report_dict['macro avg']['recall'])
        metrics_history['nature_spec'].append(fold_nat_spec)

    print()
    print("      FINAL FUSION SUMMARY (4-FOLD)      ")
    print()
    
    def p_stat(name, key):
        mean_val = np.mean(metrics_history[key])
        std_val = np.std(metrics_history[key])
        print(f"{name:<25} : {mean_val:.4f} (+/- {std_val:.4f})")

    p_stat("Global Accuracy", 'accuracy')
    p_stat("Global Macro F1", 'macro_f1')
    p_stat("Global Macro Precision", 'precision')
    p_stat("Global Macro Recall", 'recall')
    p_stat("Nature Specificity", 'nature_spec')
    print()



reconstruct_full_training_summary(results)

### Multimodal ML Predictive Utility & Statistical Significance

**Methodology:**
This section performs two non-parametric statistical tests on the aggregated out-of-fold predictions of the multimodal BiLSTM model. 95% CIs are estimated via non-parametric subject-level bootstrap resampling (2,000 iterations). Significance versus chance is assessed using a sequence-level permutation test (5,000 iterations). Note: this snippet requires the results generated by runnnig the multimodal model via the main orchestrator.


This code outputs the statistical metrics reported in manuscript Table. 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm 

def run_statistics(test_results, n_boot=2000, n_perm=5000):
    print()
    print("STATISTICAL ANALYSIS (BOOTSTRAP & PERMUTATION) ")
    print()
    
    unique_subjs = list(set([tr['subj'] for tr in test_results]))
    
    
    
 
    def calc_all_metrics(truths_list, preds_list, is_nat_list):
        flat_t = np.concatenate(truths_list)
        flat_p = np.concatenate(preds_list)
        
        acc = accuracy_score(flat_t, flat_p)
        prec, rec, f1, _ = precision_recall_fscore_support(flat_t, flat_p, average='macro', zero_division=0)
        
 
        nat_t, nat_p = [], []
        for i in range(len(is_nat_list)):
            if is_nat_list[i] == 1:
                nat_t.extend(truths_list[i])
                nat_p.extend(preds_list[i])
                
        spec = accuracy_score(nat_t, nat_p) if len(nat_t) > 0 else 1.0
        return np.array([acc, f1, prec, rec, spec])

    
    actual_truths = [tr['truths'] for tr in test_results]
    actual_preds = [tr['preds'] for tr in test_results]
    actual_nats = [tr['is_nature'] for tr in test_results]
    
    
    actual_scores = calc_all_metrics(actual_truths, actual_preds, actual_nats)
    metric_names = ["Accuracy", "Macro F1", "Macro Precision", "Macro Recall", "Nature Specificity"]
    
    
    print(f"Running {n_boot} Bootstraps (Subject-Level)...")
    

    subj_to_results = {s: [tr for tr in test_results if tr['subj'] == s] for s in unique_subjs}
    boot_distributions = []
    
    for _ in tqdm(range(n_boot), desc="Bootstrapping"):
        sampled_subjs = np.random.choice(unique_subjs, size=len(unique_subjs), replace=True) 
        b_truths, b_preds, b_nats = [], [], []
        
        for s in sampled_subjs:
            for tr in subj_to_results[s]:
                b_truths.append(tr['truths'])
                b_preds.append(tr['preds'])
                b_nats.append(tr['is_nature'])
                
        boot_distributions.append(calc_all_metrics(b_truths, b_preds, b_nats))
        
    boot_distributions = np.array(boot_distributions)
  
    print(f"Running {n_perm} Permutations (Sequence-Level)...") 
    perm_distributions = []
    
    for _ in tqdm(range(n_perm), desc="Permuting"):
        
        shuffled_truths = actual_truths.copy()
        np.random.shuffle(shuffled_truths)
        
        perm_distributions.append(calc_all_metrics(shuffled_truths, actual_preds, actual_nats))
        
    perm_distributions = np.array(perm_distributions)
    

    print()
    print(f"{'Metric':<20} | {'Score':<6} | {'95% CI':<15} | {'p-value':<8} | {'Cohen d'}")
    print()
    
    for i, name in enumerate(metric_names):
        actual = actual_scores[i]
        
       
        ci_lower, ci_upper = np.percentile(boot_distributions[:, i], [2.5, 97.5])
        
        
        perm_scores = perm_distributions[:, i]
        p_val = np.sum(perm_scores >= actual) / n_perm
        p_val_str = "<0.0002" if p_val == 0 else f"{p_val:.4f}"
        
        std_perm = np.std(perm_scores)
    
        cohens_d = (actual - np.mean(perm_scores)) / std_perm if std_perm > 0 else 0.0
        
        print(f"{name:<20} | {actual:.4f} | [{ci_lower:.3f}, {ci_upper:.3f}] | {p_val_str:<8} | {cohens_d:.2f}")
    print()


run_statistics(results)

### Multimodal Baseline: Temporal Probability Alignment

**Methodology:**
This section generates a qualitative visualization, termed a "Tinglegram," to assess the temporal accuracy of the full multimodal fusion network's predictions against the subjective ground truth for a representative well-aligned participant (Subject 27). Panel A demonstrates the ASMR condition, while Panel B shows the Nature Control condition. This code assumes the main orchestrator is run, which saves the multimodal model's test outputs.


This code outputs the Tinglegram figure for the manuscript. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from scipy.ndimage import gaussian_filter1d
import pickle
def plot_smoothed_probability_timeline(test_results, target_subj=27):
    print(f"Extracting data for Subject {target_subj}...")
   
    asmr_trial = next((item for item in test_results if item['subj'] == target_subj and item['is_nature'] == 0), None)
    nature_trial = next((item for item in test_results if item['subj'] == target_subj and item['is_nature'] == 1), None)
    
    print(f"  - ASMR Video   : {asmr_trial['vid_name']}")
    print(f"  - Nature Video : {nature_trial['vid_name']}")
    
    time_axis = np.arange(550) / 10.0  
    
    
    asmr_user_scaled = asmr_trial['raw_truths']>0
    nature_user_scaled = nature_trial['raw_truths'] / 3.0
    
    
    asmr_ai_smooth = gaussian_filter1d(asmr_trial['probs'], sigma=2)
    nature_ai_smooth = gaussian_filter1d(nature_trial['probs'], sigma=2)

    
    sns.set_theme(style="whitegrid")
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), sharex=True)
    plt.subplots_adjust(hspace=0.2)

    
    ax1.fill_between(time_axis, 0, asmr_user_scaled, color='lightgrey', step='post', label='User')
    
    ax1.plot(time_axis, asmr_ai_smooth, color='blue', linewidth=2.5, label='AI')
    
    ax1.axhline(0.5, color='red', linestyle='--', alpha=0.5, label='Threshold')
    
    ax1.set_title('ASMR', fontweight='bold')
    ax1.set_ylabel('Intensity / Probability')
    ax1.set_ylim(-0.05, 1.05)
    ax1.legend(loc='upper right', frameon=True)

   
    ax2.fill_between(time_axis, 0, nature_user_scaled, color='lightgrey', step='post', label='User')

    ax2.plot(time_axis, nature_ai_smooth, color='orange', linewidth=2.5, label='AI')
  
    ax2.axhline(0.5, color='red', linestyle='--', alpha=0.5, label='Threshold')
    
    ax2.set_title('Nature', fontweight='bold')
    ax2.set_xlabel('Time (Seconds)')
    ax2.set_ylabel('Intensity / Probability')
    ax2.set_ylim(-0.05, 1.05)
    ax2.legend(loc='upper right', frameon=True)

    
    
    plt.show()

plot_smoothed_probability_timeline(results)

### Modality Ablation Study

**Methodology:**
This section compares the full architecture against unimodal baselines (Video-only and Audio-only). 

To determine if the full fusion model provides a statistically significant improvement over the individual modalities, a **paired bootstrap significance test** (10,000 iterations) is conducted on the aggregated Macro F1-scores across the 4-fold cross-validation protocol. 
The empirical $p$-value is calculated based on the proportion of bootstrapped samples from the shifted null distribution that equal or exceed the actual observed performance gain. The hardcoded values were obtained at a representative run by the authors using random seed = 42.

This code outputs the statistical significance metrics for manuscript table.

In [ ]:
import numpy as np
from scipy import stats

def paired_bootstrap_test(model_a, model_b, n_boot=10000, seed=42):
    """
    Performs a paired bootstrap test to check if model_a is significantly better than model_b.
    """
    np.random.seed(seed)
    
   
    diffs = np.array(model_a) - np.array(model_b)
    actual_mean_diff = np.mean(diffs)
    
   
    shifted_diffs = diffs - actual_mean_diff
    
   
    boot_mean_diffs = []
    for _ in range(n_boot):
        sample = np.random.choice(shifted_diffs, size=len(shifted_diffs), replace=True)
        boot_mean_diffs.append(np.mean(sample))
        
    boot_mean_diffs = np.array(boot_mean_diffs)
    
    
    p_val = np.mean(boot_mean_diffs >= actual_mean_diff)
    
    return actual_mean_diff, p_val


multi = [0.7398, 0.7074, 0.8377, 0.5896]
vid   = [0.7400, 0.7071, 0.7433, 0.5741]
aud   = [0.7172, 0.7084, 0.7977, 0.5741]

print("  STATISTICAL COMPARISON (MULTIMODAL vs ABLATIONS) \n")


mean_diff_vid, p_boot_vid = paired_bootstrap_test(multi, vid)
_, p_ttest_vid = stats.ttest_rel(multi, vid, alternative='greater')

print("MULTIMODAL vs VIDEO-ONLY:")
print(f"  Mean Improvement : +{mean_diff_vid:.4f}")
print(f"  Bootstrap P-Value: {p_boot_vid:.4f}")
print(f"  T-Test P-Value   : {p_ttest_vid:.4f}\n")


mean_diff_aud, p_boot_aud = paired_bootstrap_test(multi, aud)
_, p_ttest_aud = stats.ttest_rel(multi, aud, alternative='greater')

print("MULTIMODAL vs AUDIO-ONLY:")
print(f"  Mean Improvement : +{mean_diff_aud:.4f}")
print(f"  Bootstrap P-Value: {p_boot_aud:.4f}")
print(f"  T-Test P-Value   : {p_ttest_aud:.4f}")


### Video-Level Macro Classification & Statistical Inference

**Methodology:**
Since physiological triggers are often episodic, to evaluate the multimodal network's macro-level utility, this section applies a peak-detection strategy to the continuous predictions. An A Priori thresholding of top 15 percent frames determines the video's classification. Bootstrapping confirms the classifications' statistical validity.



In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pickle
def run_video_level_classification(test_results, threshold_ratio=0.15):
    print()
    print("   VIDEO-LEVEL CLASSIFICATION (PEAK DETECTION STRATEGY)   ")
    print()
    
    video_truths = []
    video_preds = []
    
    frames_per_video = len(test_results[0]['preds']) 
    threshold_frames = int(frames_per_video * threshold_ratio) 
    
    print(f"Frames per video : {frames_per_video}")
    print(f"Tingle Threshold : {threshold_frames} frames ({threshold_ratio*100}%)\n")
    
    for tr in test_results:
        
        vid_truth = 0 if tr['is_nature'] == 1 else 1
        video_truths.append(vid_truth)
        
        
        predicted_tingle_frames = np.sum(tr['preds'])
        vid_pred = 1 if predicted_tingle_frames >= threshold_frames else 0
        video_preds.append(vid_pred)
        
    
    vid_acc = accuracy_score(video_truths, video_preds)
    vid_prec, vid_rec, vid_f1, _ = precision_recall_fscore_support(
        video_truths, video_preds, average='macro', zero_division=0)
        
    print(f"Video-Level Accuracy        : {vid_acc:.4f} ({vid_acc*100:.2f}%)")
    print(f"Video-Level Macro F1        : {vid_f1:.4f} ({vid_f1*100:.2f}%)")
    print(f"Video-Level Macro Precision : {vid_prec:.4f}")
    print(f"Video-Level Macro Recall    : {vid_rec:.4f}")
    
    cm = confusion_matrix(video_truths, video_preds)
    print("\nConfusion Matrix (Video-Level):")
    print(f"True Negatives (Nature correct) : {cm[0][0]}")
    print(f"False Positives (Nature wrong)  : {cm[0][1]}")
    print(f"False Negatives (ASMR missed)   : {cm[1][0]}")
    print(f"True Positives (ASMR correct)   : {cm[1][1]}")
    print()
def video_level_statistics(test_results, threshold_ratio=0.15, n_boot=2000, n_perm=5000):
    print()
    print(" VIDEO-LEVEL STATISTICAL ANALYSIS (BOOTSTRAP & PERMUTATION) ")
    print()
    
    frames_per_video = len(test_results[0]['preds'])
    threshold_frames = int(frames_per_video * threshold_ratio)
    
    
    vid_truths = []
    vid_preds = []
    
    for tr in test_results:
        truth = 0 if tr['is_nature'] == 1 else 1
        pred = 1 if np.sum(tr['preds']) >= threshold_frames else 0
        vid_truths.append(truth)
        vid_preds.append(pred)
        
    vid_truths = np.array(vid_truths)
    vid_preds = np.array(vid_preds)
    
    actual_acc = accuracy_score(vid_truths, vid_preds)
    actual_f1 = f1_score(vid_truths, vid_preds, average='macro', zero_division=0)
    
   
    print(f"Running {n_boot} Video-Level Bootstraps...")
    np.random.seed(42)
    boot_accs = []
    
    indices = np.arange(len(vid_truths))
    for _ in range(n_boot):
       
        sample_idx = np.random.choice(indices, size=len(indices), replace=True)
        sample_t = vid_truths[sample_idx]
        sample_p = vid_preds[sample_idx]
        boot_accs.append(accuracy_score(sample_t, sample_p))
        
    ci_lower, ci_upper = np.percentile(boot_accs, [2.5, 97.5])
    
 
    print(f"Running {n_perm} Video-Level Permutations...")
    perm_accs = []
    
    for _ in range(n_perm):
       
        shuffled_t = vid_truths.copy()
        np.random.shuffle(shuffled_t)
        perm_accs.append(accuracy_score(shuffled_t, vid_preds))
        
    perm_accs = np.array(perm_accs)
    p_val = np.sum(perm_accs >= actual_acc) / n_perm
    p_val_str = "< 0.0002" if p_val == 0 else f"{p_val:.4f}"
    
    print("\n--- FINAL VIDEO-LEVEL STATISTICS ---")
    print(f"Actual Accuracy : {actual_acc*100:.2f}%")
    print(f"Actual Macro F1 : {actual_f1*100:.2f}%")
    print(f"95% CI (Acc)    : [{ci_lower*100:.2f}%, {ci_upper*100:.2f}%]")
    print(f"P-Value         : {p_val_str}")
    print("-" * 70)



run_video_level_classification(results)



video_level_statistics(results)